In [1]:
from pynq.overlays.base import BaseOverlay
import time
import multiprocessing
import socket

# Load the base overlay
base = BaseOverlay("base.bit")

In [2]:
%%microblaze base.PMODA
#include "gpio.h"

// Function to set a GPIO pin as output and write a value
void write_gpio(unsigned int pin, unsigned int val) {
    if (val > 1) {
        return;
    }
    gpio pin_out = gpio_open(pin); // Open the GPIO pin
    gpio_set_direction(pin_out, GPIO_OUT); // Set direction as output
    gpio_write(pin_out, val); // Write the specified value
}

// Initialize GPIO pins for the buzzer
void init_gpio() {
    for (unsigned int pin = 0; pin < 8; pin++) {
        gpio pin_out = gpio_open(pin); // Open the GPIO pin
        gpio_set_direction(pin_out, GPIO_OUT); // Set direction as output
    }
}

In [ ]:
btns = base.buttons  # Initialize buttons

# Function to send a message to the server
def client(server_ip, server_port, action):
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(2)  # Set timeout for connection attempts
        sock.connect((server_ip, server_port))
        print(f"[INFO] Connected to server {server_ip}:{server_port}")

        print(f"[INFO] Sending action: {action}")  # ✅ Debugging print
        sock.sendall(action.encode())  # Send the action ('TONE' or 'DISCONNECT')
    except socket.error as e:
        print(f"[ERROR] Connection failed: {e}")
    finally:
        sock.close()
        print("[INFO] Socket closed")


# Button handling function
def get_btns(server_ip, server_port):
    while True:
        if btns[0].read() != 0:
            print("[INFO] Connecting to server")
            client(server_ip, server_port, 'HELLO')
        if btns[1].read() != 0:
            print("[INFO] Activate buzzer on server")
            client(server_ip, server_port, 'TONE')
        if btns[2].read() != 0:
            print("[INFO] Disconnecting from server")
            client(server_ip, server_port, 'DISCONNECT')
        if btns[3].read() != 0:
            print("[INFO] Only buttons 1, 2, and 3 are operational")
        
        time.sleep(0.1)  # Debounce delay

if __name__ == "__main__":
    server_ip = "192.168.2.99"  # Change to PYNQ's IP if testing across devices
    port = 9999
    get_btns(server_ip, port)


[INFO] Connecting to server
[INFO] Connected to server 192.168.2.99:9999
[INFO] Sending action: HELLO
[INFO] Socket closed
[INFO] Activate buzzer on server
[INFO] Connected to server 192.168.2.99:9999
[INFO] Sending action: TONE
[INFO] Socket closed
[INFO] Connecting to server
[INFO] Connected to server 192.168.2.99:9999
[INFO] Sending action: HELLO
[INFO] Socket closed
[INFO] Connecting to server
[INFO] Connected to server 192.168.2.99:9999
[INFO] Sending action: HELLO
[INFO] Socket closed
[INFO] Disconnecting from server
[INFO] Connected to server 192.168.2.99:9999
[INFO] Sending action: DISCONNECT
[INFO] Socket closed
[INFO] Connecting to server
[INFO] Connected to server 192.168.2.99:9999
[INFO] Sending action: HELLO
[INFO] Socket closed
[INFO] Connecting to server
[INFO] Connected to server 192.168.2.99:9999
[INFO] Sending action: HELLO
[INFO] Socket closed
[INFO] Activate buzzer on server
[INFO] Connected to server 192.168.2.99:9999
[INFO] Sending action: TONE
[INFO] Socket close